# Bond Futures Samples

This notebook demonstrates how to access and use the financial-contracts end point of Instrument Pricing Analytics (IPA) in the Refinitiv Data Platform.

##### Learn more

Please visit the [Refinitiv Developer Community Portal](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-bond-future-contracts) to access the API documentation and much more.

You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

##### Getting Help and Support

If you have any questions regarding the API usage, please post them on the [Refinitiv Data Platform Q&A Forum](https://community.developers.refinitiv.com/spaces/231/index.html). The Refinitiv Developer Community will be very pleased to help you. 

#### Import the library and connect to the Refinitv Data Platform

This sample works in both desktop and non-desktop environment. The appropriate session can be created based on the constant variable IPA_ENVIRONMENT, that is set by default to DESKTOP for execution in the Refinitiv codebook application. To run this sample in a non-desktop environment, you may set the variable to NON-DESKTOP.

In [1]:
import refinitiv.dataplatform as rdp
import pandas as pd

# Set IPA_ENVIRONMENT to NON-DESKTOP to use this example in a non desktop environment
#IPA_ENVIRONMENT = "NON-DESKTOP"
IPA_ENVIRONMENT = "DESKTOP"

if (IPA_ENVIRONMENT == "DESKTOP"):
    session = rdp.DesktopSession(app_key='DEFAULT_CODE_BOOK_APP_KEY')
else:
    %run ".\..\..\credentials.ipynb"
    session = rdp.PlatformSession(APP_KEY, rdp.GrantPassword(RDP_LOGIN, RDP_PASSWORD))

session.open()
endpoint = rdp.Endpoint(session, "https://api.refinitiv.com/data/quantitative-analytics/v1/financial-contracts")
print("session opened and end-point set")

session opened and end-point set


### Computing Bond futures 

This sample provides the main future information and analytics. The 'fields' requested are:
- the descriptive parameters of the bond future (InstrumentCode,InstrumentDescription, CouponRatePercent,EndDate)
- The Cheapest to Deliver (CheapestToDeliver), its conversion factor (ConversionFactor), the invoice price (InvoicePrice), and the CTD yield implied from this invoice price (ImpliedYieldPercent)
- the sensitivities of the bond future (ModifiedDuration, DV01Bp, Convexity)
- the error information (ErrorCode, ErrorMessage)


In [3]:
request_body = {
    "fields": ["InstrumentCode","InstrumentDescription", "CouponRatePercent","EndDate",
               "CheapestToDeliver", "ConversionFactor", "InvoicePrice", "ImpliedYieldPercent",
               "ModifiedDuration", "DV01Bp", "Convexity",
               "ErrorCode", "ErrorMessage"],
    
    "universe": [
        {
            "instrumentType": "BondFuture",
            "instrumentDefinition": {
                "instrumentCode": "FOATc3"
            }
            
        }
    ],
    
    "pricingParameters": {
    },

    "outputs": ["Headers","Data"]
}

response = endpoint.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body)

print(response.data.raw)

headers_name = [h['name'] for h in response.data.raw['headers']]
df = pd.DataFrame(data=response.data.raw['data'], columns=headers_name)
df

{'headers': [{'type': 'String', 'name': 'InstrumentCode'}, {'type': 'String', 'name': 'InstrumentDescription'}, {'type': 'Float', 'name': 'CouponRatePercent'}, {'type': 'DateTime', 'name': 'EndDate'}, {'type': 'String', 'name': 'CheapestToDeliver'}, {'type': 'Float', 'name': 'ConversionFactor'}, {'type': 'Float', 'name': 'InvoicePrice'}, {'type': 'Float', 'name': 'ImpliedYieldPercent'}, {'type': 'Float', 'name': 'ModifiedDuration'}, {'type': 'Float', 'name': 'DV01Bp'}, {'type': 'Float', 'name': 'Convexity'}, {'type': 'String', 'name': 'ErrorCode'}, {'type': 'String', 'name': 'ErrorMessage'}], 'data': [['FOATc3', 'FOAT DEC1/d', 6.0, '2021-12-08T00:00:00Z', 'FR0012993103=', 0.682206, 108.7777467, 0.544854375903666, 8.30580700500276, 13.2436099231859, 85.1351768428435, '', '']]}


,InstrumentCode,InstrumentDescription,CouponRatePercent,EndDate,CheapestToDeliver,ConversionFactor,InvoicePrice,ImpliedYieldPercent,ModifiedDuration,DV01Bp,Convexity,ErrorCode,ErrorMessage
0,FOATc3,FOAT DEC1/d,6.0,2021-12-08T00:00:00Z,FR0012993103=,0.682206,108.777747,0.544854,8.305807,13.24361,85.135177,,
